<a href="https://colab.research.google.com/github/Nami-59/KNN/blob/main/Python_kNN/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [5]:
data = pd.read_csv('ISiT(Answers).csv', sep=';', encoding='utf-8')

In [6]:
data = data.iloc[:,1:]

# При нормализации ошибка прогноза становится увеличивается, т.е. алгоритм работает хуже
#data_y = data['Кофе или чай?']
#data = data.iloc[:,:9]
#data = (data-data.min())/(data.max()-data.min())

In [ ]:
#data['Кофе или чай?'] = data_y
#data

In [7]:
# разделение выборки на тестовую и обучающую
size=26
data_train = data.iloc[:size,:]
x_train = data_train.values

data_test = data.iloc[size:,:]
x_test = data_test.values


In [8]:
def minKowski_distance(data_1, data_2, p): # метрика Миньковского
  distance = 0
  for i in range(len(data_1) - 1):
    distance += (abs(data_1[i] - data_2[i])) ** p
  return distance ** (1/p)

In [91]:
def get_neighbors(test, train, p, k):

    distances = [(train[i][-1], minKowski_distance(train[i], test, p))
                  for i in range(len(train))] # расстояние тестовых до каждого обучающего элемента
    distances.sort(key=lambda elem: elem[1])
    k = min(k, len(distances))
    neighbors = [distances[i][0] for i in range(k)] # ближайшие соседи, до которых расстояние мин., из них и предсказывается значение

    return neighbors

In [10]:
def prediction(neighbors): # предсказание
  count = {}
  for instance in neighbors: # считается кол-во наибольших соседей из классов (пример: если мин. расстояние до 3х чаев и 2х кофе, то предскажется чай)
     if instance in count:
        count[instance] +=1
     else :
        count[instance] = 1
  result = max(count.items(), key=lambda x: x[1])[0]
  return result

In [11]:
def error(test, test_pred): # ошибка прогноза
    correct = 0
    for i in range(len(test)):
        if test[i][-1] == test_pred[i]:
            correct += 1
    return 1-(correct/len(test))

In [92]:
def predict(p, k, x_test, x_train): # вычисление ошибки и прогноз
  predictions = []
  for x in range(len(x_test)):
    neighbors = get_neighbors(x_test[x], x_train, p, k)
    result = prediction(neighbors)
    predictions.append(result)
   # print(f'Предсказание = {result}, Исходное = {x_test[x][-1]}')

  error_1 = error(x_test, predictions)
  return error_1, predictions

In [93]:
def res(p, k, x_test, x_train): # находит оптимальные k по мин. ошибке прогноза для каждой p
  k_opt = []
  error_opt = []
  k_opt_new = 0
  error_opt_new = 0
  for i in range(1,p):
    k_opt.append(k_opt_new)
    error_opt.append(error_opt_new)
  # print()
 #  print(f'Метрика порядка p: {i}') # если хотите просмотреть всю прогонку
#   print()
    error_opt_new = 1.1
    for j in range(1,k):
   #    print(f'Общие соседи k: {j}') # если хотите посмотреть всю прогонку каждого k
        error_k, pred = predict(i,j,x_test,x_train)
        if error_opt_new > error_k: # мин ошибка для нахождения мин k
          error_opt_new = error_k
          k_opt_new = j
    #   print(f'Error: {error_k}') # если хотите посмотреть всю прогонку каждой ошибки для k и p

  k_opt.append(k_opt_new)
  k_opt.pop(0)
  error_opt.append(error_opt_new)
  error_opt.pop(0)

  return k_opt, error_opt

In [78]:
def optimization(p, k, x_test, x_train): # выбор оптимального p при минимальной ошибке
  k_opt, error_opt = res(p, k, x_test, x_train)
#  print(f'k оптим. соседей для каждого p : {k_opt}')
 # print(f'Ошибка при данном k : {error_opt}')
  index = error_opt.index(min(error_opt))
  p_opt_min = index + 1
  k_opt_min = k_opt[index]
  error_opt_min = error_opt[index]

  return p_opt_min, k_opt_min, error_opt_min

In [98]:
p = 10
k = 10
p_opt_min, k_opt_min, error_opt_min = optimization(p, k, x_test, x_train)
print('Оптимальная система: ')
print()
print(f'Оптимальное наименьшее p: {p_opt_min}')
print(f'Оптимальное k соседей: {k_opt_min}')
print(f'Ошибка прогноза: {error_opt_min}')

Оптимальная система: 

Оптимальное наименьшее p: 2
Оптимальное k соседей: 4
Ошибка прогноза: 0.1333333333333333


In [95]:
print('Манхэттенская метрика p = 1, k = 4')
print()
error_ma, pred = predict(1, 4, x_test, x_train)

for i in range(0,len(pred)):
  print(f'Предсказание = {pred[i]}, Исходное = {x_test[i][9]}')

print(f'Error: {error_ma}')

Манхэттенская метрика p = 1, k = 4

Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Кофе
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Кофе
Предсказание = Чай, Исходное = nan
Error: 0.19999999999999996


In [96]:
print('Евклидовая метрика p = 2, k = 4') # она же наша оптимальная
print()
error_ev, pred = predict(2, 4, x_test, x_train)

for i in range(0,len(pred)):
  print(f'Предсказание = {pred[i]}, Исходное = {x_test[i][9]}')

print(f'Error: {error_ev}')

Евклидовая метрика p = 2, k = 4

Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Кофе, Исходное = Кофе
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Кофе
Предсказание = Чай, Исходное = nan
Error: 0.1333333333333333


In [97]:
print('Метрика Миньковского p = 3, k = 4')
print()
error_mi, pred = predict(3, 4, x_test, x_train)

for i in range(0,len(pred)):
  print(f'Предсказание = {pred[i]}, Исходное = {x_test[i][9]}')

print(f'Error: {error_mi}')

Метрика Миньковского p = 3, k = 4

Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Кофе, Исходное = Кофе
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Чай
Предсказание = Чай, Исходное = Кофе
Предсказание = Чай, Исходное = nan
Error: 0.1333333333333333
